# 모델, 데이터 불러와서 결괏값 출력까지
- API제작을 위한 공유용 노트

## 코드 작성

In [2]:
pwd

'C:\\Users\\user\\Desktop\\개발\\git_code\\2차_미니프로젝트\\Model\\notebooks\\질병예측모델_240520'

In [4]:
# from flask import Flask, request, jsonify
import pickle
from gensim.models import Word2Vec
import numpy as np


# 모델과 데이터 로드
model = Word2Vec.load("./model/240517_final.model")

# disease_list.pkl파일은 공유하지 않음
with open('./disease_list.pkl', 'rb') as f:
    disease_list = pickle.load(f)

def predict(input_symptoms):

    # 질병 리스트
    diseases = disease_list
    
    # 각 증상에 대해 가장 유사한 질병과 그 유사도 찾기
    symptom_to_diseases_similarity = {}
    for symptom in input_symptoms:
        try:
            similar_words = model.wv.most_similar(symptom, topn=300)
            symptom_to_diseases_similarity[symptom] = [(word, similarity) for word, similarity in similar_words if word in diseases]
        except KeyError:
            pass
    
    # 질병별로 증상과의 유사도 및 일치하는 증상 수 저장
    disease_similarity_info = {}
    for symptom, diseases_similarities in symptom_to_diseases_similarity.items():
        for disease, similarity in diseases_similarities:
            if disease in disease_similarity_info:
                disease_similarity_info[disease]['count'] += 1
                disease_similarity_info[disease]['similarities'].append(similarity)
            else:
                disease_similarity_info[disease] = {'count': 1, 'similarities': [similarity]}
    
    # 증상이 2개 이상 일치하는 질병 필터링 및 평균 유사도 계산
    if len(symptom_to_diseases_similarity.keys()) == 1:
        filtered_diseases = [(disease, info['count'], np.mean(info['similarities']), info['similarities']) 
                         for disease, info in disease_similarity_info.items() if info['count'] >= 1]
    else:
        filtered_diseases = [(disease, info['count'], np.mean(info['similarities']), info['similarities']) 
                         for disease, info in disease_similarity_info.items() if info['count'] >= 2]
    
    # 증상 일치 수 및 평균 유사도에 따라 정렬
    filtered_diseases.sort(key=lambda x: (x[1], x[2]), reverse=True)
    
    # 상위 5개 출력
    top_diseases = filtered_diseases[:5]
    
    result = [{"질병": disease, "일치하는 증상 수": count, "평균 유사도": avg_similarity, "증상별 유사도": similarities} 
              for disease, count, avg_similarity, similarities in top_diseases]
    
    return result


## 예측하기

In [15]:
predict(['기침', '떨림(진전)', '손떨림', '치아 흔들림', '객혈', '온몸이 떨림', '사래걸림'])

[{'질병': '동맥색전증 및 혈전증(Arterial embolism and thrombosis)',
  '일치하는 증상 수': 5,
  '평균 유사도': 0.9958996534347534,
  '증상별 유사도': [0.9976767301559448,
   0.9933732748031616,
   0.9974362850189209,
   0.9968411922454834,
   0.9941707849502563]},
 {'질병': '외상성 치매 (Traumatic dementia)',
  '일치하는 증상 수': 5,
  '평균 유사도': 0.9958613395690918,
  '증상별 유사도': [0.9976925849914551,
   0.993455708026886,
   0.9974370002746582,
   0.9968697428703308,
   0.9938516616821289]},
 {'질병': '기분부전증(Dysthymia)',
  '일치하는 증상 수': 5,
  '평균 유사도': 0.9958328127861023,
  '증상별 유사도': [0.9978289008140564,
   0.9930130839347839,
   0.9973030686378479,
   0.997012734413147,
   0.9940062761306763]},
 {'질병': '비정형 파킨슨 증후군(Atypical Parkinsonism)',
  '일치하는 증상 수': 5,
  '평균 유사도': 0.9958099484443664,
  '증상별 유사도': [0.9979000687599182,
   0.9929311275482178,
   0.997713565826416,
   0.9968559741973877,
   0.9936490058898926]},
 {'질병': '망상 장애(Delusional disorder)',
  '일치하는 증상 수': 5,
  '평균 유사도': 0.9956621408462525,
  '증상별 유사도': [0.9971085786819458,